# Generating sequences on the fly on submission

In [1]:
import numpy as np
import datetime
import time
import edd.experiments as edde
from edd.backend import IBMQBackend
from edd.circuit import IBMQDdCircuit
from edd.pulse import IBMQDdSchedule
from edd.data import IBMQData
import edd.workflow as wf

from matplotlib import rc
rc("text", usetex=False)

# Submission Settings

In [2]:
basis = "g_basis"
# qubits on which to encode and to apply DD on
qubit = 2
encoding_qubits = qubit
dd_qubits = qubit

#sched_settings = [offset, sym, delay, d_label, basis, encoding_qubits, dd_qubits]

backend_name = 'bogota'
# deprecated way to load in backend when my account info was hard-coded
#backend = IBMQBackend(f'ibmq_{backend_name}', "Nic")
# use this with your own info...
backend = IBMQBackend(name, hub, group, project, token) 
max_time = 4.65 # in micro seconds

def ns_time_to_dt(phys_time, dt):
    '''converts time from physical (ns) to dt normalized time'''
    return (phys_time * 1e-9) / dt

IBM Backend ibmq_bogota loaded at date/time: 2022-03-03 11:43:27


In [3]:
backend.backend.remaining_jobs_count()

20

## Generate list of best sequences below maximum allotted time

In [13]:
max_time = 75

def add_seq(sched, seq, qubit, delay, sym, reps):
    if "uddx" in seq:
        seq_method = "add_uddx"
        ord_n = int(seq[4:])
        getattr(sched, seq_method)(ord_n, qubit, reps, delay, sym)
    elif "qdd" in seq:
        seq_method = "add_qdd"
        _, ord_n, ord_m = seq.split("_")
        ord_n = int(ord_n)
        ord_m = int(ord_m)
        getattr(sched, seq_method)(ord_n, ord_m, qubit, reps, delay, sym)
    elif "ur" in seq and "x" not in seq and "y" not in seq:
        seq_method = "add_ur"
        ord_n = int(seq[2:])
        getattr(sched, seq_method)(ord_n, qubit, reps, delay, sym)
    else:
        getattr(sched, f"add_{seq}")(qubit, reps, delay, sym)
        
    return sched

def find_largest_delays(seq, sched):
    """
    Find the largest asymmetric and symmetric delays
    for a given sequence. That is, the delay
    that can only fit a single repetition of
    the sequence.
    """
    # first find d_asym
    d_asym = int(ns_time_to_dt(max_time * 1000, sched.dt))
    # confine d_asym to a reasonably sized interval
    lower = 0
    upper = d_asym
    found_interval = False
    while found_interval is False:
        sched.reset()
        add_seq(sched, seq, qubit, d_asym, False, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            upper = d_asym
            d_asym = int(d_asym / 2)
        else:
            found_interval = True
            lower = d_asym
    # find smallest d_asym that works
    for d_asym in range(lower, upper + 1):
        sched.reset()
        add_seq(sched, seq, qubit, d_asym, False, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            break
    d_asym -= 1
    
    # now find d_sym using same methodology 
    d_sym = int(ns_time_to_dt(max_time * 1000, sched.dt))
    # confine d_sym to a reasonably sized interval
    lower = 0
    upper = d_sym
    found_interval = False
    while found_interval is False:
        sched.reset()
        add_seq(sched, seq, qubit, d_sym, True, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            upper = d_sym
            d_sym = int(d_sym / 2)
        else:
            found_interval = True
            lower = d_sym
    # find smallest d_sym that works
    for d_sym in range(lower, upper + 1):
        sched.reset()
        add_seq(sched, seq, qubit, d_sym, True, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            break
    d_sym -= 1
    
            
    return d_asym, d_sym

In [28]:
current_seqs = ['ur20', 'purex', 'xy4']
max_asym_delay = {}
max_sym_delay = {}
sched = IBMQDdSchedule(backend, "g_basis")
for seq in current_seqs:
    if seq != "free":
        d_asym, d_sym = find_largest_delays(seq, sched)
        max_asym_delay[seq] = d_asym
        max_sym_delay[seq] = d_sym
    else:
        max_asym_delay["free"] = 0
        max_sym_delay["free"] = 0

In [30]:
max_sym_delay

{'ur20': 8357, 'purex': 84295, 'xy4': 42107}

In [26]:
sched = IBMQDdSchedule(backend, "g_basis")
list_of_seqs = ['ur20']
delay = 12703
sym = True
satisfies_time = []
for seq in list_of_seqs:
    sched.reset()
    add_seq(sched, seq, qubit, delay, sym, 1)
    T = sched.get_phys_time() / 1000
    print(T)
    if T <= max_time:
        satisfies_time.append(seq)
        
satisfies_time

75.0051111111111


[]

In [ ]:
# dictionary of max delays for 75 \mu s since takes to long to generate by looking
max_asym_delay = {'ur20': 17594, 'qdd_3_5': 12703, 'purex': 168590, 'xy4': 84215}
max_sym_delay = {'ur20': 8357, 'qdd_3_5': 6351, 'purex': 84295, 'xy4': 42107}

In [6]:
list_of_short_seqs = ["ur20", "qdd_1_2", "purex", "xy4"]
len(list_of_short_seqs)

4

# Load in Haar random U3 parameters and choose how many states to test

In [11]:
# load in Haar random states and choose how many to use
haar_u3_params = np.load("../../u3_list.npy")
N = 25

## Compute delay list for each sequence/ symmetry

In [12]:
asym_delays = {}
sym_delays = {}
num_delays = 8

for seq in list_of_short_seqs:
    max_dasym = max_asym_delay[seq]
    delays = [int(x) for x in np.linspace(0, max_dasym, num_delays)]
    asym_delays[seq] = delays
    
    max_dsym = max_sym_delay[seq]
    delays = [int(x) for x in np.linspace(0, max_dsym, num_delays)]
    sym_delays[seq] = delays

# Functions to Create all the schedules to submit

In [13]:
def generate_schedules(seq, backend, basis, delays, qubit, sym, nHaar):
    """
    Generates schedules for [seq] that all last for max_time
    but have different pulse delays.
    """
    sched = IBMQDdSchedule(backend, basis)
    sched_list = []
    for d in delays:
        sched.reset()
        add_seq(sched, seq, qubit, d, sym, 1)
        T = sched.get_phys_time() / 1000
        max_reps = int(np.floor(max_time / T))
        T_diff = int(np.ceil(ns_time_to_dt((max_time - max_reps * T) * 1000, sched.dt)))

        for n in range(nHaar):
            u3_params = haar_u3_params[n]
            name = f"seq_{seq}_delay_{d}_state_{n}"
            seq_sched = IBMQDdSchedule(backend, basis, name=name)
            seq_sched.add_u3(qubit, *u3_params[0])
            add_seq(seq_sched, seq, qubit, d, sym, max_reps)
            sched.add_pause(qubit, T_diff)
            seq_sched.add_u3(qubit, *u3_params[1])
            seq_sched.diagnose_acquire_constraint(qubit)
            seq_sched.add_measurement(qubit, 0)
            sched_list.append(seq_sched)

    return sched_list

# Submit the jobs and save as dictionary

In [14]:
now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
backend_fname = backend_name + "_" + now
backend.save_backend_props(backend_fname + "_haarJob_properties.txt")

asym_job_dict = {}
asym_failures = []

sym_job_dict = {}
sym_failures = []

for seq in list_of_short_seqs:
    if seq == "xy4":
        basis = "c_basis"
    else:
        basis = "g_basis"
    print(f"Trying to submit {seq}")
    # try asymmetric job 1
    try:
        sym = False
        delays = asym_delays[seq][0:4]
        scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
        job_id = f"sym_{sym}_seq_{seq}_job_1"
        job = backend.submit_job(scheds, job_id, num_shots=8192)
        asym_job_dict[job_id] = job
        print("Asym1 Success")
    except:
        asym_failures.append(seq)
        print("Asym1 Failure")
    # try asymmetric job 2
    try:
        sym = False
        delays = asym_delays[seq][4:8]
        scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
        job_id = f"sym_{sym}_seq_{seq}_job_2"
        job = backend.submit_job(scheds, job_id, num_shots=8192)
        asym_job_dict[job_id] = job
        print("Asym2 Success")
    except:
        asym_failures.append(seq)
        print("Asym2 Failure")
    # try symmetric job 1
    try:
        sym = True
        delays = sym_delays[seq][0:4]
        scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
        job_id = f"sym_{sym}_seq_{seq}_job_1"
        job = backend.submit_job(scheds, job_id, num_shots=8192)
        sym_job_dict[job_id] = job
        print("Sym1 Success")
    except:
        sym_failures.append(seq)
        print("Sym1 Failure")
    # try symmetric job 2
    try:
        sym = True
        delays = sym_delays[seq][4:8]
        scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
        job_id = f"sym_{sym}_seq_{seq}_job_2"
        job = backend.submit_job(scheds, job_id, num_shots=8192)
        sym_job_dict[job_id] = job
        print("Sym2 Success")
    except:
        sym_failures.append(seq)
        print("Sym2 Failure")

Trying to submit ur20


/Users/worknic/opt/anaconda3/envs/edd18Aug2021/lib/python3.9/site-packages/qiskit/compiler/assembler.py:436: RuntimeWarning: Dynamic rep rates are supported on this backend. 'rep_delay' will be used instead of 'rep_time'.
  warnings.warn(


Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit qdd_1_2
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit purex
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit xy4
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success


# Retrieve Jobs and Wrangle Data

### Wrangle and Save asymmetric results

In [15]:
asym_results = {}
asym_not_finished = []
for seq in list_of_short_seqs:
    try:
        key = f"sym_False_seq_{seq}_job_1"
        result = asym_job_dict[key].result()
        asym_results[f"{seq}_job_1"] = result
        key = f"sym_False_seq_{seq}_job_2"
        result = asym_job_dict[key].result()
        asym_results[f"{seq}_job_2"] = result
    except:
        asym_not_finished.append(seq)
        
# save the raw results
for key, result in asym_results.items():
    data = IBMQData(result)
    fname = backend_fname + "_" + key + "_asym_rawData"
    data.save_raw_data(fname)

### Wrangle and Save symmetric results

In [16]:
sym_results = {}
sym_not_finished = []
for seq in list_of_short_seqs:
    try:
        key = f"sym_True_seq_{seq}_job_1"
        result = sym_job_dict[key].result()
        sym_results[f"{seq}_job_1"] = result
        key = f"sym_True_seq_{seq}_job_2"
        result = sym_job_dict[key].result()
        sym_results[f"{seq}_job_2"] = result
    except:
        sym_not_finished.append(seq)
        
# save the raw results
for key, result in sym_results.items():
    data = IBMQData(result)
    fname = backend_fname + "_" + key + "_sym_rawData"
    data.save_raw_data(fname)

## Put in a loop so don't have to check continuously

In [13]:
current_seqs = ['free', 'purex']

In [14]:
for j in range(1):
    print(f"Loop count: {j}")
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    backend_fname = backend_name + "_" + now
    backend.save_backend_props(backend_fname + "_haarJob_properties.txt")

    asym_job_dict = {}
    asym_failures = []
    sym_job_dict = {}
    sym_failures = []
    for seq in current_seqs:
        if seq == "xy4":
            basis = "c_basis"
        else:
            basis = "g_basis"
        print(f"Trying to submit {seq}")
        # try asymmetric job
        try:
            sym = False
            delays = asym_delays[seq]
            scheds = generate_schedules(seq, backend, basis, delays, qubit, sym)
            job_id = f"sym_{sym}_seq_{seq}"
            job = backend.submit_job(scheds, job_id, num_shots=8192)
            asym_job_dict[job_id] = job
            print("Success")
        except:
            asym_failures.append(seq)
            print("Failure")
        # try symmetric job
        try:
            sym = True
            delays = sym_delays[seq]
            scheds = generate_schedules(seq, backend, basis, delays, qubit, sym)
            job_id = f"sym_{sym}_seq_{seq}"
            job = backend.submit_job(scheds, job_id, num_shots=8192)
            sym_job_dict[job_id] = job
            print("Success")
        except:
            sym_failures.append(seq)
            print("Failure")
            
    time.sleep(60 * 60)
            
    retrieved = False
    while retrieved is False:
        asym_results = {}
        asym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_False_seq_{seq}"
                result = asym_job_dict[key].result()
                asym_results[seq] = result
            except:
                asym_not_finished.append(seq)
                
        if asym_not_finished == []:
            retrieved = True
        else:
            time.sleep(60 * 30)
            
    # save the raw results
    for seq, result in asym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + seq + "_asym_rawData"
        data.save_raw_data(fname)
        
    retrieved = False
    while retrieved is False:
        sym_results = {}
        sym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_True_seq_{seq}"
                result = sym_job_dict[key].result()
                sym_results[seq] = result
            except:
                sym_not_finished.append(seq)
                
        if sym_not_finished == []:
            retrieved = True
        else:
            time.sleep(60 * 30)
            
    # save the raw results
    for seq, result in sym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + seq + "_sym_rawData"
        data.save_raw_data(fname)

Loop count: 0
Trying to submit free
Success
Success
Trying to submit purex
Success
Success


## A self contained way to change desired sequences and duration then run

In [14]:
def add_seq(sched, seq, qubit, delay, sym, reps):
    if "uddx" in seq:
        seq_method = "add_uddx"
        ord_n = int(seq[4:])
        getattr(sched, seq_method)(ord_n, qubit, reps, delay, sym)
    elif "qdd" in seq:
        seq_method = "add_qdd"
        _, ord_n, ord_m = seq.split("_")
        ord_n = int(ord_n)
        ord_m = int(ord_m)
        getattr(sched, seq_method)(ord_n, ord_m, qubit, reps, delay, sym)
    elif "ur" in seq and "x" not in seq and "y" not in seq:
        seq_method = "add_ur"
        ord_n = int(seq[2:])
        getattr(sched, seq_method)(ord_n, qubit, reps, delay, sym)
    else:
        getattr(sched, f"add_{seq}")(qubit, reps, delay, sym)
        
    return sched

def find_largest_delays(seq, sched):
    """
    Find the largest asymmetric and symmetric delays
    for a given sequence. That is, the delay
    that can only fit a single repetition of
    the sequence.
    """
    # first find d_asym
    d_asym = int(ns_time_to_dt(max_time * 1000, sched.dt))
    # confine d_asym to a reasonably sized interval
    lower = 0
    upper = d_asym
    found_interval = False
    while found_interval is False:
        sched.reset()
        add_seq(sched, seq, qubit, d_asym, False, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            upper = d_asym
            d_asym = int(d_asym / 2)
        else:
            found_interval = True
            lower = d_asym
    # find smallest d_asym that works
    for d_asym in range(lower, upper + 1):
        sched.reset()
        add_seq(sched, seq, qubit, d_asym, False, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            break
    d_asym -= 1
    
    # now find d_sym using same methodology 
    d_sym = int(ns_time_to_dt(max_time * 1000, sched.dt))
    # confine d_sym to a reasonably sized interval
    lower = 0
    upper = d_sym
    found_interval = False
    while found_interval is False:
        sched.reset()
        add_seq(sched, seq, qubit, d_sym, True, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            upper = d_sym
            d_sym = int(d_sym / 2)
        else:
            found_interval = True
            lower = d_sym
    # find smallest d_sym that works
    for d_sym in range(lower, upper + 1):
        sched.reset()
        add_seq(sched, seq, qubit, d_sym, True, 1)
        T = sched.get_phys_time() / 1000
        if T > max_time:
            break
    d_sym -= 1
    
            
    return d_asym, d_sym

current_seqs = []
max_asym_delay = {}
max_sym_delay = {}
sched = IBMQDdSchedule(backend, "g_basis")
for seq in current_seqs:
    if seq != "free":
        d_asym, d_sym = find_largest_delays(seq, sched)
        max_asym_delay[seq] = d_asym
        max_sym_delay[seq] = d_sym
    else:
        max_asym_delay["free"] = 0
        max_sym_delay["free"] = 0

In [15]:
def generate_schedules(seq, backend, basis, delays, qubit, sym, nHaar):
    """
    Generates schedules for [seq] that all last for max_time
    but have different pulse delays.
    """
    sched = IBMQDdSchedule(backend, basis)
    sched_list = []
    for d in delays:
        sched.reset()
        add_seq(sched, seq, qubit, d, sym, 1)
        T = sched.get_phys_time() / 1000
        max_reps = int(np.floor(max_time / T))
        T_diff = int(np.ceil(ns_time_to_dt((max_time - max_reps * T) * 1000, sched.dt)))

        for n in range(nHaar):
            u3_params = haar_u3_params[n]
            name = f"seq_{seq}_delay_{d}_state_{n}"
            seq_sched = IBMQDdSchedule(backend, basis, name=name)
            seq_sched.add_u3(qubit, *u3_params[0])
            add_seq(seq_sched, seq, qubit, d, sym, max_reps)
            sched.add_pause(qubit, T_diff)
            seq_sched.add_u3(qubit, *u3_params[1])
            seq_sched.diagnose_acquire_constraint(qubit)
            seq_sched.add_measurement(qubit, 0)
            sched_list.append(seq_sched)

    return sched_list

In [ ]:
# load in backend again
backend = IBMQBackend(f'ibmq_{backend_name}', "Vinay")
# set max time in micro-seconds
#max_time = 4.65
#time_label = "short"
# choose sequences to explore
#current_seqs = ["ur20", "qdd_1_2", "purex", "xy4"]

max_time = 75
time_label = "long"
# choose sequences to explore
current_seqs = ['qdd_3_5', 'purex', 'xy4', 'free', 'ur20']
# load in Haar random states and choose how many to use
haar_u3_params = np.load("../../u3_list.npy")
N = 25
# number of delay points
num_delays = 8

if max_time == 75:
    # dictionary of max delays for 75 \mu s since takes to long to generate by looking
    max_asym_delay = {'ur20': 17594, 'qdd_3_5': 12703, 'purex': 168590, 'xy4': 84215, 'free': 0}
    max_sym_delay = {'ur20': 8357, 'qdd_3_5': 6351, 'purex': 84295, 'xy4': 42107, 'free': 0}
else:
    max_asym_delay = {}
    max_sym_delay = {}
    sched = IBMQDdSchedule(backend, "g_basis")
    for seq in current_seqs:
        if seq != "free":
            d_asym, d_sym = find_largest_delays(seq, sched)
            max_asym_delay[seq] = d_asym
            max_sym_delay[seq] = d_sym
        else:
            max_asym_delay["free"] = 0
            max_sym_delay["free"] = 0
    
asym_delays = {}
sym_delays = {}
for seq in current_seqs:
    if seq != "free":
        max_dasym = max_asym_delay[seq]
        delays = [int(x) for x in np.linspace(0, max_dasym, num_delays)]
        asym_delays[seq] = delays
        max_dsym = max_sym_delay[seq]
        delays = [int(x) for x in np.linspace(0, max_dsym, num_delays)]
        sym_delays[seq] = delays
    else:
        asym_delays["free"] = [0]
        sym_delays["free"] = [0]
        
        
for j in range(5):
    print(f"Loop count: {j}")
    # load in backend and get calibration data
    backend = IBMQBackend(f'ibmq_{backend_name}', "Vinay")
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    backend_fname = backend_name + "_" + now
    backend.save_backend_props(backend_fname + "_haarJob_properties.txt")

    # submit jobs
    asym_job_dict = {}
    asym_failures = []
    sym_job_dict = {}
    sym_failures = []
    for seq in current_seqs:
        if seq == "xy4":
            basis = "c_basis"
        else:
            basis = "g_basis"
        print(f"Trying to submit {seq}")
        # if sequence is free evolution do special thing
        if seq == "free":
            # try asymmetric job
            try:
                sym = False
                delays = asym_delays[seq]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym Success")
            except:
                asym_failures.append(seq)
                print("Asym1 Failure")
            # try symmetric job
            try:
                sym = True
                delays = sym_delays[seq]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym Success")
            except:
                sym_failures.append(seq)
                print("Sym Failure")
        # otherwise, do normal thing
        else:
            # try asymmetric job 1
            try:
                sym = False
                delays = asym_delays[seq][0:4]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym1 Success")
            except:
                asym_failures.append(seq)
                print("Asym1 Failure")
            # try asymmetric job 2
            try:
                sym = False
                delays = asym_delays[seq][4:8]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_2"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym2 Success")
            except:
                asym_failures.append(seq)
                print("Asym2 Failure")
            # try symmetric job 1
            try:
                sym = True
                delays = sym_delays[seq][0:4]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym1 Success")
            except:
                sym_failures.append(seq)
                print("Sym1 Failure")
            # try symmetric job 2
            try:
                sym = True
                delays = sym_delays[seq][4:8]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_2"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym2 Success")
            except:
                sym_failures.append(seq)
                print("Sym2 Failure")
            
    time.sleep(60 * 30)

    # load in data and save it
    retrieved = 0
    while retrieved < 2:
        asym_results = {}
        asym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_False_seq_{seq}_job_1"
                result = asym_job_dict[key].result()
                asym_results[f"{seq}_job_1"] = result
                if seq != "free":
                    key = f"sym_False_seq_{seq}_job_2"
                    result = asym_job_dict[key].result()
                    asym_results[f"{seq}_job_2"] = result
            except:
                asym_not_finished.append(seq)

        if asym_not_finished == []:
            retrieved = 2
        else:
            retrieved += 1
            time.sleep(60 * 6)

    # save the raw results
    for key, result in asym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + key + f"_asym_tLab_{time_label}_rawData"
        data.save_raw_data(fname)

    retrieved = 0
    while retrieved < 2:
        sym_results = {}
        sym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_True_seq_{seq}_job_1"
                result = sym_job_dict[key].result()
                sym_results[f"{seq}_job_1"] = result
                if seq != "free":
                    key = f"sym_True_seq_{seq}_job_2"
                    result = sym_job_dict[key].result()
                    sym_results[f"{seq}_job_2"] = result
            except:
                sym_not_finished.append(seq)

        if sym_not_finished == []:
            retrieved = 2
        else:
            retrieved += 1
            time.sleep(60 * 6)

    # save the raw results
    for key, result in sym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + key + f"_sym_tLab_{time_label}_rawData"
        data.save_raw_data(fname)# load in backend again

ibmqfactory.load_account:WARNING:2022-03-03 12:14:49,795: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-03 12:14:55
Loop count: 0


ibmqfactory.load_account:WARNING:2022-03-03 12:14:56,469: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-03 12:15:01
Trying to submit qdd_3_5


/Users/worknic/opt/anaconda3/envs/edd18Aug2021/lib/python3.9/site-packages/qiskit/compiler/assembler.py:436: RuntimeWarning: Dynamic rep rates are supported on this backend. 'rep_delay' will be used instead of 'rep_time'.
  warnings.warn(


...

ibmqfactory.load_account:WARNING:2022-03-04 11:07:17,139: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-04 11:07:21
Trying to submit qdd_3_5
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit purex
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit xy4
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success
Trying to submit free
Asym Success
Sym Success
Trying to submit ur20
Asym1 Success
Asym2 Success
Sym1 Success
Sym2 Success


Loop count: 2


ibmqfactory.load_account:WARNING:2022-03-05 16:11:02,001: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-05 16:11:07
Trying to submit qdd_3_5
Asym1 Success


In [5]:
# load in backend again
backend = IBMQBackend(f'ibmq_{backend_name}', "Vinay")
# set max time in micro-seconds
max_time = 75
time_label = "long"
# choose sequences to explore
current_seqs = ['qdd_3_5', 'purex', 'xy4', 'free', 'ur20']
# load in Haar random states and choose how many to use
haar_u3_params = np.load("../../u3_list.npy")
N = 25
# number of delay points
num_delays = 8

if max_time == 75:
    # dictionary of max delays for 75 \mu s since takes to long to generate by looking
    max_asym_delay = {'ur20': 17594, 'qdd_3_5': 12703, 'purex': 168590, 'xy4': 84215, 'free': 0}
    max_sym_delay = {'ur20': 8357, 'qdd_3_5': 6351, 'purex': 84295, 'xy4': 42107, 'free': 0}
else:
    max_asym_delay = {}
    max_sym_delay = {}
    sched = IBMQDdSchedule(backend, "g_basis")
    for seq in current_seqs:
        if seq != "free":
            d_asym, d_sym = find_largest_delays(seq, sched)
            max_asym_delay[seq] = d_asym
            max_sym_delay[seq] = d_sym
        else:
            max_asym_delay["free"] = 0
            max_sym_delay["free"] = 0
    
asym_delays = {}
sym_delays = {}
for seq in current_seqs:
    if seq != "free":
        max_dasym = max_asym_delay[seq]
        delays = [int(x) for x in np.linspace(0, max_dasym, num_delays)]
        asym_delays[seq] = delays
        max_dsym = max_sym_delay[seq]
        delays = [int(x) for x in np.linspace(0, max_dsym, num_delays)]
        sym_delays[seq] = delays
    else:
        asym_delays["free"] = [0]
        sym_delays["free"] = [0]
        
        
for j in range(2):
    print(f"Loop count: {j}")
    # load in backend and get calibration data
    backend = IBMQBackend(f'ibmq_{backend_name}', "Vinay")
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    backend_fname = backend_name + "_" + now
    backend.save_backend_props(backend_fname + "_haarJob_properties.txt")

    # submit jobs
    asym_job_dict = {}
    asym_failures = []
    sym_job_dict = {}
    sym_failures = []
    for seq in current_seqs:
        if seq == "xy4":
            basis = "c_basis"
        else:
            basis = "g_basis"
        print(f"Trying to submit {seq}")
        # if sequence is free evolution do special thing
        if seq == "free":
            # try asymmetric job
            try:
                sym = False
                delays = asym_delays[seq]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym Success")
            except:
                asym_failures.append(seq)
                print("Asym1 Failure")
            # try symmetric job
            try:
                sym = True
                delays = sym_delays[seq]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym Success")
            except:
                sym_failures.append(seq)
                print("Sym Failure")
        # otherwise, do normal thing
        else:
            # try asymmetric job 1
            try:
                sym = False
                delays = asym_delays[seq][0:4]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym1 Success")
            except:
                asym_failures.append(seq)
                print("Asym1 Failure")
            # try asymmetric job 2
            try:
                sym = False
                delays = asym_delays[seq][4:8]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_2"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                asym_job_dict[job_id] = job
                print("Asym2 Success")
            except:
                asym_failures.append(seq)
                print("Asym2 Failure")
            # try symmetric job 1
            try:
                sym = True
                delays = sym_delays[seq][0:4]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_1"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym1 Success")
            except:
                sym_failures.append(seq)
                print("Sym1 Failure")
            # try symmetric job 2
            try:
                sym = True
                delays = sym_delays[seq][4:8]
                scheds = generate_schedules(seq, backend, basis, delays, qubit, sym, N)
                job_id = f"sym_{sym}_seq_{seq}_job_2"
                job = backend.submit_job(scheds, job_id, num_shots=8192)
                sym_job_dict[job_id] = job
                print("Sym2 Success")
            except:
                sym_failures.append(seq)
                print("Sym2 Failure")
            
    time.sleep(60 * 30)

    # load in data and save it
    retrieved = False
    while retrieved is False:
        asym_results = {}
        asym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_False_seq_{seq}_job_1"
                result = asym_job_dict[key].result()
                asym_results[f"{seq}_job_1"] = result
                if seq != "free":
                    key = f"sym_False_seq_{seq}_job_2"
                    result = asym_job_dict[key].result()
                    asym_results[f"{seq}_job_2"] = result
            except:
                asym_not_finished.append(seq)

        if asym_not_finished == []:
            retrieved = True
        else:
            time.sleep(60 * 6)

    # save the raw results
    for key, result in asym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + key + f"_asym_tLab_{time_label}_rawData"
        data.save_raw_data(fname)

    retrieved = False
    while retrieved is False:
        sym_results = {}
        sym_not_finished = []
        for seq in current_seqs:
            try:
                key = f"sym_True_seq_{seq}_job_1"
                result = sym_job_dict[key].result()
                sym_results[f"{seq}_job_1"] = result
                if seq != "free":
                    key = f"sym_True_seq_{seq}_job_2"
                    result = sym_job_dict[key].result()
                    sym_results[f"{seq}_job_2"] = result
            except:
                sym_not_finished.append(seq)

        if sym_not_finished == []:
            retrieved = True
        else:
            time.sleep(60 * 6)

    # save the raw results
    for key, result in sym_results.items():
        data = IBMQData(result)
        fname = backend_fname + "_" + key + f"_sym_tLab_{time_label}_rawData"
        data.save_raw_data(fname)

ibmqfactory.load_account:WARNING:2022-03-03 11:43:59,178: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-03 11:44:04
Loop count: 0


ibmqfactory.load_account:WARNING:2022-03-03 11:44:05,300: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_bogota loaded at date/time: 2022-03-03 11:44:10
Trying to submit qdd_3_5
Asym1 Failure
Asym2 Failure
Sym1 Failure
Sym2 Failure
Trying to submit purex
Asym1 Failure
Asym2 Failure
Sym1 Failure
Sym2 Failure
Trying to submit xy4
Asym1 Failure
Asym2 Failure
Sym1 Failure
Sym2 Failure
Trying to submit free
Asym1 Failure
Sym Failure
Trying to submit ur20
Asym1 Failure
Asym2 Failure
Sym1 Failure
Sym2 Failure


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/worknic/opt/anaconda3/envs/edd18Aug2021/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/2w/_70fq7w11c36fkhz8pnqp8gw0000gp/T/ipykernel_34657/489839968.py", line 142, in <module>
    time.sleep(60 * 30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/worknic/opt/anaconda3/envs/edd18Aug2021/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/worknic/opt/anaconda3/envs/edd18Aug2021/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    retur

TypeError: object of type 'NoneType' has no len()

In [6]:
job

NameError: name 'job' is not defined

In [85]:
asym_job_dict['sym_False_seq_qdd_3_5_job_1'].status()

<JobStatus.QUEUED: 'job is queued'>

In [86]:
asym_job_dict.keys()

dict_keys(['sym_False_seq_ur20_job_1', 'sym_False_seq_ur20_job_2', 'sym_False_seq_qdd_3_5_job_1', 'sym_False_seq_qdd_3_5_job_2', 'sym_False_seq_purex_job_1', 'sym_False_seq_purex_job_2', 'sym_False_seq_xy4_job_1', 'sym_False_seq_xy4_job_2', 'sym_False_seq_free_job_1'])

In [100]:
asym_job_dict['sym_False_seq_ur20_job_1'].status().name

'DONE'

In [82]:
# load in data and save it
asym_results = {}
asym_not_finished = []
for seq in ["qdd_3_5", "purex", "xy4", "free"]:
    try:
        key = f"sym_False_seq_{seq}_job_1"
        print(key)
        result = asym_job_dict[key].result()
        asym_results[f"{seq}_job_1"] = result
        print("finito")
        if seq != "free":
            key = f"sym_False_seq_{seq}_job_2"
            print(key)
            result = asym_job_dict[key].result()
            asym_results[f"{seq}_job_2"] = result
    except:
        asym_not_finished.append(seq)

sym_False_seq_qdd_3_5_job_1
sym_False_seq_purex_job_1
sym_False_seq_xy4_job_1
sym_False_seq_free_job_1


In [81]:
# save the raw results
for key, result in asym_results.items():
    data = IBMQData(result)
    fname = backend_fname + "_" + key + f"_asym_tLab_{time_label}_rawData"
    data.save_raw_data(fname)

In [76]:
sym_results = {}
sym_not_finished = []
for seq in ['ur20']:
    try:
        key = f"sym_True_seq_{seq}_job_1"
        result = sym_job_dict[key].result()
        sym_results[f"{seq}_job_1"] = result
        if seq != "free":
            key = f"sym_True_seq_{seq}_job_2"
            result = sym_job_dict[key].result()
            sym_results[f"{seq}_job_2"] = result
    except:
        sym_not_finished.append(seq)

In [77]:
# save the raw results
for key, result in sym_results.items():
    data = IBMQData(result)
    fname = backend_fname + "_" + key + f"_sym_tLab_{time_label}_rawData"
    data.save_raw_data(fname)

## Cancelling a job (just in case)

In [72]:
backend.backend.remaining_jobs_count()

20

In [5]:
for j in range(13):
    backend.backend.active_jobs()[0].cancel()